In [1]:
import pandas as pd
import traintools
from trainconstants import *
from collections import defaultdict
import networkx as nx
from typing import List, Tuple

TrainStop = Tuple[int, int]

In [2]:
def attempt_solution(G: nx.DiGraph) -> Tuple[int, defaultdict, defaultdict]:
    """Find the path for a train such that it makes the most useful trips.
    Repeat until there are no more trains needed.

    For this solution the endpoints still need fixing.
    """

    def do_bookkeeping(path: List[TrainStop]) -> None:
        """Update the variables that keep track of the total number of trains 
        and where they start and end.
        """
        nonlocal all_trains, starting_trains, ending_trains, number_of_trains

        all_trains.append(path)
        starting_trains[path[0][0]].append(path)
        ending_trains[path[-1][0]].append(path)
        number_of_trains += 1

    # set up variables to keep track of the number of trains and at which stations the trains start and end.
    starting_trains, ending_trains, number_of_trains = defaultdict(list), defaultdict(list), 0
    all_trains = []

    # find the longest path in the graph
    path = nx.dag_longest_path(G, weight='min_trains')

    # repeat until the longest path consists of only one node
    while nx.path_weight(G, path, weight="min_trains") > 0:
        
        # cut off empty trips from the front
        to_be_removed = []
        for index, current_stop in enumerate(path[:-1]):
            next_stop = path[index + 1]
            if G[current_stop][next_stop]['min_trains'] == 0:
                to_be_removed.append(current_stop)
            else: 
                break
        
        # cut off empty trains from the back
        for index, next_stop in enumerate(path[-1:0:-1]):
            current_stop = path[-(index + 2)]
            if G[current_stop][next_stop]['min_trains'] == 0:
                to_be_removed.append(next_stop)
            else: 
                break
        
        for stop in set(to_be_removed):
            path.remove(stop)

        do_bookkeeping(path)
        
        # go through the longest path and pick up the maximal number of passengers on the way
        for index, current_stop in enumerate(path[:-1]):
            next_stop = path[index + 1]
            if G[current_stop][next_stop]['min_trains'] > 0:
                G[current_stop][next_stop]['min_trains'] -= 1
    
        # recompute the longest path 
        path = nx.dag_longest_path(G, weight='min_trains')
        
        

    return(
        all_trains, 
        number_of_trains, 
        starting_trains, ending_trains,
        G
        )

In [3]:
df = traintools.read_schedule("datasets/nsdata1.txt")
G = traintools.graph_from_schedule(df, TYPE_3_TRAIN)
all_trains, number_of_trains, starting_trains, ending_trains, G = attempt_solution(G.copy())
print(number_of_trains, starting_trains, ending_trains)

24 defaultdict(<class 'list'>, {3: [[(3, 329), (2, 388), (2, 389), (1, 458), (1, 475), (2, 538), (2, 542), (3, 581), (3, 583), (3, 590), (3, 593), (2, 632), (2, 634), (1, 698), (1, 716), (2, 778), (2, 782), (3, 821), (3, 823), (3, 830), (3, 833), (2, 872), (2, 875), (1, 938), (1, 956), (2, 1018), (2, 1021), (3, 1063), (3, 1065), (3, 1070), (3, 1073), (2, 1112), (2, 1114), (1, 1178), (1, 1196), (2, 1258), (2, 1262), (3, 1301), (3, 1303), (3, 1313), (2, 1352), (2, 1354), (2, 1378), (2, 1382), (3, 1434)], [(3, 329), (2, 388), (2, 389), (2, 420), (3, 460), (3, 463), (4, 518), (4, 536), (3, 590), (3, 593), (2, 632), (2, 634), (1, 698), (1, 716), (2, 778), (2, 782), (3, 821), (3, 823), (3, 830), (3, 833), (2, 872), (2, 875), (1, 938), (1, 956), (2, 1018), (2, 1021), (3, 1063), (3, 1065), (4, 1120), (4, 1136), (3, 1190), (3, 1193), (2, 1232), (2, 1235), (2, 1258), (2, 1262), (3, 1301), (3, 1303), (3, 1313), (2, 1352), (2, 1354), (1, 1418)], [(3, 403), (2, 446), (2, 452), (1, 518), (1, 536), (

In [4]:
for train in all_trains:
    for index, stop in enumerate(train[:-1]):
        print(stop, f"---<<{G[stop][train[index + 1]]['min_trains']}>>",end='---')
    print(train[-1])

(3, 329) ---<<0>>---(2, 388) ---<<0>>---(2, 389) ---<<0>>---(1, 458) ---<<0>>---(1, 475) ---<<0>>---(2, 538) ---<<0>>---(2, 542) ---<<0>>---(3, 581) ---<<0>>---(3, 583) ---<<0>>---(3, 590) ---<<0>>---(3, 593) ---<<0>>---(2, 632) ---<<0>>---(2, 634) ---<<0>>---(1, 698) ---<<0>>---(1, 716) ---<<0>>---(2, 778) ---<<0>>---(2, 782) ---<<0>>---(3, 821) ---<<0>>---(3, 823) ---<<0>>---(3, 830) ---<<0>>---(3, 833) ---<<0>>---(2, 872) ---<<0>>---(2, 875) ---<<0>>---(1, 938) ---<<0>>---(1, 956) ---<<0>>---(2, 1018) ---<<0>>---(2, 1021) ---<<0>>---(3, 1063) ---<<0>>---(3, 1065) ---<<0>>---(3, 1070) ---<<0>>---(3, 1073) ---<<0>>---(2, 1112) ---<<0>>---(2, 1114) ---<<0>>---(1, 1178) ---<<0>>---(1, 1196) ---<<0>>---(2, 1258) ---<<0>>---(2, 1262) ---<<0>>---(3, 1301) ---<<0>>---(3, 1303) ---<<0>>---(3, 1313) ---<<0>>---(2, 1352) ---<<0>>---(2, 1354) ---<<0>>---(2, 1378) ---<<0>>---(2, 1382) ---<<0>>---(3, 1434)
(4, 330) ---<<0>>---(3, 395) ---<<0>>---(3, 403) ---<<0>>---(2, 446) ---<<0>>---(2, 452) --

In [5]:
starting_stops = tuple(traintools.find_starting_trainstops(G))
ending_stops = tuple(traintools.find_ending_trainstops(G))

print(starting_stops)

for paths in starting_trains.values():
    for path in paths:
        print(tuple(trainstop for trainstop in starting_stops if nx.has_path(G, trainstop, path[0])))

((2, 331), (3, 329), (4, 330), (1, 399))
((3, 329),)
((3, 329),)
((3, 329), (4, 330))
((2, 331), (3, 329), (4, 330))
((3, 329), (4, 330))
((2, 331), (3, 329), (4, 330))
((2, 331), (3, 329), (4, 330))
((2, 331), (3, 329), (4, 330), (1, 399))
((4, 330),)
((4, 330),)
((4, 330),)
((4, 330),)
((4, 330),)
((4, 330),)
((4, 330),)
((2, 331),)
((2, 331), (3, 329))
((2, 331), (3, 329), (4, 330))
((2, 331), (3, 329), (4, 330), (1, 399))
((2, 331), (3, 329), (4, 330), (1, 399))
((2, 331), (1, 399))
((2, 331), (1, 399))
((2, 331), (3, 329), (1, 399))
((2, 331), (3, 329), (1, 399))


In [6]:
for paths in ending_trains.values():
    for path in paths:
        print(tuple(trainstop for trainstop in ending_stops if nx.has_path(G, path[-1], trainstop)))

((3, 1434),)
((3, 1434),)
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438),)
((2, 1438), (3, 1434))
((2, 1438), (3, 1434))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((1, 1418),)
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((4, 1358),)
((4, 1358),)
((4, 1358),)
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((4, 1358),)
((4, 1358),)
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))
((2, 1438), (3, 1434), (4, 1358), (1, 1418))


In [7]:
def attempt_solution_correct_endpoints(G: nx.DiGraph, start: TrainStop = None) -> Tuple[int, defaultdict, defaultdict]:
    """Attempt at a solution where the end points don't need to be fixed.
    """

    def do_bookkeeping(path: List[TrainStop]) -> None:
        """Update the variables that keep track of the total number of trains 
        and where they start and end.
        """
        nonlocal all_trains, starting_trains, ending_trains, number_of_trains

        all_trains.append(path)
        starting_trains[path[0][0]].append(path)
        ending_trains[path[-1][0]].append(path)
        number_of_trains += 1


    def cut_empty_trips(path: List[TrainStop]) -> List[TrainStop]:
        """Cut the empty trips of the from of the front and back of a path and return it.
        """
        
        to_be_removed = []
        for index, current_stop in enumerate(path[:-1]):
            next_stop = path[index + 1]
            if G[current_stop][next_stop]['min_trains'] == 0:
                to_be_removed.append(current_stop)
            else: 
                break
        
        # cut off empty trains from the back
        for index, next_stop in enumerate(path[-1:0:-1]):
            current_stop = path[-(index + 2)]
            if G[current_stop][next_stop]['min_trains'] == 0:
                to_be_removed.append(next_stop)
            else: 
                break
        
        for stop in set(to_be_removed):
            path.remove(stop)

        return path

    
    def add_temp_weight(node: TrainStop) -> None:
        """Add temp node to fix starting point of the longest path.
        """
        nonlocal G

        G.add_node('temp')
        G.add_edge('temp', node, min_trains=1000)


    def remove_temp_weight() -> None:
        """Remove temp node to get the 'true' graph back.
        """

        nonlocal G

        G.remove_node('temp')
        


    # set up variables to keep track of the number of trains and at which stations the trains start and end.
    starting_trains, ending_trains, number_of_trains = defaultdict(list), defaultdict(list), 0
    all_trains = []

    starting_stops = {stop[0]: stop for stop in traintools.find_starting_trainstops(G)}

    # find the longest path in the graph
    if start is None:
        path = nx.dag_longest_path(G, weight='min_trains')
        end = path[-1]
    else:
        add_temp_weight(start)
        path = nx.dag_longest_path(G, weight='min_trains')[1:]
        remove_temp_weight()  
        end = path[-1]
    

    # repeat until the longest path consists of only one node
    loops = 0
    while G.size(weight='min_trains') > 0:
        
        path = cut_empty_trips(path)

        do_bookkeeping(path)
        
        # go through the longest path and pick up the maximal number of passengers on the way
        for index, current_stop in enumerate(path[:-1]):
            next_stop = path[index + 1]
            if G[current_stop][next_stop]['min_trains'] > 0:
                G[current_stop][next_stop]['min_trains'] -= 1
    

        # recompute the longest path 
        start = starting_stops[path[-1][0]]
        if start == end:
            temp_dict = {key: len(value)-ending_trains[key] for key, value in starting_trains.items()}
            new_start = (key for key, value in temp_dict if value==max(temp_dict.values()))
            start = starting_stops[new_start]
            add_temp_weight(start)
            path = nx.dag_longest_path(G, weight='min_trains')[1:]
            remove_temp_weight()
        else:
            add_temp_weight(start)
            path = nx.dag_longest_path(G, weight='min_trains')[1:]
            remove_temp_weight()  
        
        loops += 1
        

    return(
        all_trains, 
        number_of_trains, 
        starting_trains, 
        ending_trains,
        G
        )

In [8]:
df = traintools.read_schedule("datasets/nsdata1.txt")
G = traintools.graph_from_schedule(df, TYPE_3_TRAIN)
all_trains, number_of_trains, starting_trains, ending_trains, G = attempt_solution_correct_endpoints(G)

In [ ]:
print(number_of_trains)

24


In [ ]:
for start in traintools.find_starting_trainstops(G):
    df = traintools.read_schedule("datasets/nsdata1.txt")
    G = traintools.graph_from_schedule(df, TYPE_3_TRAIN)
    all_trains, number_of_trains, starting_trains, ending_trains, G = attempt_solution_correct_endpoints(G)
    print(number_of_trains)
    print({key: len(value) for key, value in starting_trains.items()})
    print({key: len(value) for key, value in ending_trains.items()})
    print(G.size(weight='min_trains'))
    final_path = nx.dag_longest_path(G, weight='min_trains')
    for index, stop in enumerate(final_path[:-1]):
        print(stop, f"---<<{G[stop][final_path[index + 1]]['min_trains']}>>", end='---')
    print(final_path[-1])
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

KeyboardInterrupt: 